# Συνεδρία 5 – Πολυ-Πράκτορας Ορχηστρωτής

Παρουσιάζει μια απλή διαδικασία δύο πρακτόρων (Ερευνητής -> Συντάκτης) χρησιμοποιώντας το Foundry Local.


### Επεξήγηση: Εγκατάσταση Εξαρτήσεων
Εγκαθιστά το `foundry-local-sdk` και το `openai`, που απαιτούνται για την πρόσβαση σε τοπικά μοντέλα και την ολοκλήρωση συνομιλιών. Ιδενποτέντο.


# Σενάριο
Υλοποιεί ένα ελάχιστο μοτίβο ορχηστρωτή δύο πρακτόρων:
- **Πράκτορας Ερευνητής** συλλέγει συνοπτικά γεγονότα σε μορφή κουκκίδων
- **Πράκτορας Συντάκτης** ξαναγράφει για εκτελεστική σαφήνεια

Δείχνει κοινή μνήμη ανά πράκτορα, διαδοχική μεταβίβαση ενδιάμεσων αποτελεσμάτων και μια απλή λειτουργία αγωγού. Επεκτάσιμο σε περισσότερους ρόλους (π.χ., Κριτικός, Επαληθευτής) ή παράλληλα παρακλάδια.

**Μεταβλητές Περιβάλλοντος:**
- `FOUNDRY_LOCAL_ALIAS` - Προεπιλεγμένο μοντέλο προς χρήση (προεπιλογή: phi-4-mini)
- `AGENT_MODEL_PRIMARY` - Κύριο μοντέλο πράκτορα (υπερισχύει του ALIAS)
- `AGENT_MODEL_EDITOR` - Μοντέλο πράκτορα συντάκτη (προεπιλογή: κύριο)

**Αναφορά SDK:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local

**Πώς λειτουργεί:**
1. **FoundryLocalManager** ξεκινά αυτόματα την υπηρεσία Foundry Local
2. Κατεβάζει και φορτώνει το καθορισμένο μοντέλο (ή χρησιμοποιεί την αποθηκευμένη έκδοση)
3. Παρέχει ένα συμβατό με OpenAI endpoint για αλληλεπίδραση
4. Κάθε πράκτορας μπορεί να χρησιμοποιεί διαφορετικό μοντέλο για εξειδικευμένες εργασίες
5. Η ενσωματωμένη λογική επαναπροσπάθειας χειρίζεται προσωρινές αποτυχίες με ευελιξία

**Βασικά Χαρακτηριστικά:**
- ✅ Αυτόματη ανακάλυψη και αρχικοποίηση υπηρεσίας
- ✅ Διαχείριση κύκλου ζωής μοντέλου (λήψη, αποθήκευση, φόρτωση)
- ✅ Συμβατότητα με OpenAI SDK για οικεία API
- ✅ Υποστήριξη πολλαπλών μοντέλων για εξειδίκευση πρακτόρων
- ✅ Ανθεκτικός χειρισμός σφαλμάτων με λογική επαναπροσπάθειας
- ✅ Τοπική πρόβλεψη (χωρίς ανάγκη για cloud API)


In [16]:
# Install dependencies
!pip install -q foundry-local-sdk openai

### Επεξήγηση: Βασικές Εισαγωγές & Τύποι
Παρουσιάζει dataclasses για την αποθήκευση μηνυμάτων του πράκτορα και υποδείξεις τύπων για σαφήνεια. Εισάγει τον Foundry Local manager + OpenAI client για επόμενες ενέργειες του πράκτορα.


In [17]:
from dataclasses import dataclass, field
from typing import List
import os
from foundry_local import FoundryLocalManager
from openai import OpenAI

### Επεξήγηση: Αρχικοποίηση Μοντέλου (Μοτίβο SDK)
Χρησιμοποιεί το Foundry Local Python SDK για αξιόπιστη διαχείριση μοντέλων:
- **FoundryLocalManager(alias)** - Εκκινεί αυτόματα την υπηρεσία και φορτώνει το μοντέλο μέσω του alias
- **get_model_info(alias)** - Μετατρέπει το alias σε συγκεκριμένο ID μοντέλου
- **manager.endpoint** - Παρέχει το endpoint της υπηρεσίας για τον OpenAI client
- **manager.api_key** - Παρέχει το API key (προαιρετικό για τοπική χρήση)
- Υποστηρίζει ξεχωριστά μοντέλα για διαφορετικούς agents (κύριος vs editor)
- Ενσωματωμένη λογική επαναπροσπάθειας με εκθετική καθυστέρηση για ανθεκτικότητα
- Επαλήθευση σύνδεσης για να διασφαλιστεί ότι η υπηρεσία είναι έτοιμη

**Βασικό Μοτίβο SDK:**
```python
manager = FoundryLocalManager(alias)
model_info = manager.get_model_info(alias)
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key)
```

**Διαχείριση Κύκλου Ζωής:**
- Οι managers αποθηκεύονται παγκοσμίως για σωστή εκκαθάριση
- Κάθε agent μπορεί να χρησιμοποιεί διαφορετικό μοντέλο για εξειδίκευση
- Αυτόματη ανακάλυψη υπηρεσιών και διαχείριση σύνδεσης
- Ομαλή επαναπροσπάθεια με εκθετική καθυστέρηση σε περιπτώσεις αποτυχίας

Αυτό διασφαλίζει τη σωστή αρχικοποίηση πριν ξεκινήσει η ορχήστρωση των agents.

**Αναφορά:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local


In [18]:
import time

# Environment configuration
PRIMARY_ALIAS = os.getenv('AGENT_MODEL_PRIMARY', os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-4-mini'))
EDITOR_ALIAS = os.getenv('AGENT_MODEL_EDITOR', PRIMARY_ALIAS)

# Store managers globally for proper lifecycle management
primary_manager = None
editor_manager = None

def init_model(alias: str, max_retries: int = 3):
    """Initialize Foundry Local manager with retry logic.
    
    Args:
        alias: Model alias to initialize
        max_retries: Number of retry attempts with exponential backoff
    
    Returns:
        Tuple of (manager, client, model_id, endpoint)
    """
    delay = 2.0
    last_err = None
    
    for attempt in range(1, max_retries + 1):
        try:
            print(f"[Init] Starting Foundry Local for '{alias}' (attempt {attempt}/{max_retries})...")
            
            # Initialize manager - this starts the service and loads the model
            manager = FoundryLocalManager(alias)
            
            # Get model info to retrieve the actual model ID
            model_info = manager.get_model_info(alias)
            model_id = model_info.id
            
            # Create OpenAI client with manager's endpoint
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key or 'not-needed'
            )
            
            # Verify the connection with a simple test
            models = client.models.list()
            print(f"[OK] Initialized '{alias}' -> {model_id} at {manager.endpoint}")
            
            return manager, client, model_id, manager.endpoint
            
        except Exception as e:
            last_err = e
            if attempt < max_retries:
                print(f"[Retry {attempt}/{max_retries}] Failed to init '{alias}': {e}")
                print(f"[Retry] Waiting {delay:.1f}s before retry...")
                time.sleep(delay)
                delay *= 2
            else:
                print(f"[ERROR] Failed to initialize '{alias}' after {max_retries} attempts")
    
    raise RuntimeError(f"Failed to initialize '{alias}' after {max_retries} attempts: {last_err}")

# Initialize primary model (for researcher)
print(f"\n{'='*80}")
print(f"Initializing Primary Model: {PRIMARY_ALIAS}")
print('='*80)
primary_manager, primary_client, PRIMARY_MODEL_ID, primary_endpoint = init_model(PRIMARY_ALIAS)

# Initialize editor model (may be same as primary)
if EDITOR_ALIAS != PRIMARY_ALIAS:
    print(f"\n{'='*80}")
    print(f"Initializing Editor Model: {EDITOR_ALIAS}")
    print('='*80)
    editor_manager, editor_client, EDITOR_MODEL_ID, editor_endpoint = init_model(EDITOR_ALIAS)
else:
    print(f"\n[Info] Editor using same model as primary")
    editor_manager = primary_manager
    editor_client, EDITOR_MODEL_ID = primary_client, PRIMARY_MODEL_ID
    editor_endpoint = primary_endpoint

print(f"\n{'='*80}")
print(f"[Configuration Summary]")
print('='*80)
print(f"  Primary Agent:")
print(f"    - Alias: {PRIMARY_ALIAS}")
print(f"    - Model: {PRIMARY_MODEL_ID}")
print(f"    - Endpoint: {primary_endpoint}")
print(f"\n  Editor Agent:")
print(f"    - Alias: {EDITOR_ALIAS}")
print(f"    - Model: {EDITOR_MODEL_ID}")
print(f"    - Endpoint: {editor_endpoint}")
print('='*80)



Initializing Primary Model: phi-4-mini
[Init] Starting Foundry Local for 'phi-4-mini' (attempt 1/3)...
[OK] Initialized 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4 at http://127.0.0.1:59959/v1

Initializing Editor Model: gpt-oss-20b
[Init] Starting Foundry Local for 'gpt-oss-20b' (attempt 1/3)...
[OK] Initialized 'gpt-oss-20b' -> gpt-oss-20b-cuda-gpu:1 at http://127.0.0.1:59959/v1

[Configuration Summary]
  Primary Agent:
    - Alias: phi-4-mini
    - Model: Phi-4-mini-instruct-cuda-gpu:4
    - Endpoint: http://127.0.0.1:59959/v1

  Editor Agent:
    - Alias: gpt-oss-20b
    - Model: gpt-oss-20b-cuda-gpu:1
    - Endpoint: http://127.0.0.1:59959/v1


### Επεξήγηση: Κλάσεις Agent & Memory
Ορίζει την ελαφριά `AgentMsg` για καταχωρήσεις μνήμης και την `Agent` που περιλαμβάνει:
- **Ρόλος συστήματος** - Η προσωπικότητα και οι οδηγίες του Agent
- **Ιστορικό μηνυμάτων** - Διατηρεί το πλαίσιο της συνομιλίας
- **Μέθοδος act()** - Εκτελεί ενέργειες με σωστή διαχείριση σφαλμάτων

Ο agent μπορεί να χρησιμοποιήσει διαφορετικά μοντέλα (κύριο vs editor) και διατηρεί απομονωμένο πλαίσιο ανά agent. Αυτό το μοτίβο επιτρέπει:
- Επίμονη μνήμη κατά τη διάρκεια ενεργειών
- Ευέλικτη ανάθεση μοντέλου ανά agent
- Απομόνωση και ανάκτηση από σφάλματα
- Εύκολη αλυσίδωση και ορχήστρωση


In [19]:
@dataclass
class AgentMsg:
    role: str
    content: str

@dataclass
class Agent:
    name: str
    system: str
    client: OpenAI = None  # Allow per-agent client assignment
    model_id: str = None   # Allow per-agent model
    memory: List[AgentMsg] = field(default_factory=list)

    def _history(self):
        """Return chat history in OpenAI messages format including system + memory."""
        msgs = [{'role': 'system', 'content': self.system}]
        for m in self.memory[-6:]:  # Keep last 6 messages to avoid context overflow
            msgs.append({'role': m.role, 'content': m.content})
        return msgs

    def act(self, prompt: str, temperature: float = 0.4, max_tokens: int = 300):
        """Send a prompt, store user + assistant messages in memory, and return assistant text.
        
        Args:
            prompt: User input/task for the agent
            temperature: Sampling temperature (0.0-1.0)
            max_tokens: Maximum tokens to generate
        
        Returns:
            Assistant response text
        """
        # Use agent-specific client/model or fall back to primary
        client_to_use = self.client or primary_client
        model_to_use = self.model_id or PRIMARY_MODEL_ID
        
        self.memory.append(AgentMsg('user', prompt))
        
        try:
            # Build messages including system prompt and history
            messages = self._history() + [{'role': 'user', 'content': prompt}]
            
            resp = client_to_use.chat.completions.create(
                model=model_to_use,
                messages=messages,
                max_tokens=max_tokens,
                temperature=temperature,
            )
            
            # Validate response
            if not resp.choices:
                raise RuntimeError("No completion choices returned")
            
            out = resp.choices[0].message.content or ""
            
            if not out:
                raise RuntimeError("Empty response content")
            
        except Exception as e:
            out = f"[ERROR:{self.name}] {type(e).__name__}: {str(e)}"
            print(f"[Agent Error] {self.name}: {type(e).__name__}: {str(e)}")
        
        self.memory.append(AgentMsg('assistant', out))
        return out

print("[INFO] Agent classes initialized with Foundry SDK support")
print(f"[INFO] Using OpenAI SDK version: {OpenAI.__module__}")


[INFO] Agent classes initialized with Foundry SDK support
[INFO] Using OpenAI SDK version: openai


### Επεξήγηση: Οργανωμένη Ροή Εργασιών
Δημιουργεί δύο εξειδικευμένους πράκτορες:
- **Ερευνητής**: Χρησιμοποιεί το κύριο μοντέλο, συλλέγει πραγματολογικές πληροφορίες
- **Συντάκτης**: Μπορεί να χρησιμοποιήσει ξεχωριστό μοντέλο (αν έχει ρυθμιστεί), βελτιώνει και ξαναγράφει

Η συνάρτηση `pipeline`:
1. Ο Ερευνητής συλλέγει ακατέργαστες πληροφορίες
2. Ο Συντάκτης τις βελτιώνει σε έτοιμο εκτελεστικό αποτέλεσμα
3. Επιστρέφει τόσο τα ενδιάμεσα όσο και τα τελικά αποτελέσματα

Αυτό το μοτίβο επιτρέπει:
- Εξειδίκευση μοντέλων (διαφορετικά μοντέλα για διαφορετικούς ρόλους)
- Βελτίωση ποιότητας μέσω επεξεργασίας πολλών σταδίων
- Ιχνηλασιμότητα της μετατροπής πληροφοριών
- Εύκολη επέκταση σε περισσότερους πράκτορες ή παράλληλη επεξεργασία


In [ ]:
# Create specialized agents with optional model assignment
researcher = Agent(
    name='Researcher',
    system='You collect concise factual bullet points.',
    client=primary_client,
    model_id=PRIMARY_MODEL_ID
)

editor = Agent(
    name='Editor',
    system='You rewrite content for clarity and an executive, action-focused tone.',
    client=editor_client,
    model_id=EDITOR_MODEL_ID
)

def pipeline(q: str, verbose: bool = True):
    """Execute multi-agent pipeline: Researcher -> Editor.
    
    Args:
        q: User question/task
        verbose: Print intermediate outputs
    
    Returns:
        Dictionary with research, final outputs, and metadata
    """
    if verbose:
        print(f"[Pipeline] Question: {q}\n")
    
    # Stage 1: Research
    if verbose:
        print("[Stage 1: Research]")
    research = researcher.act(q)
    if verbose:
        print(f"Output: {research[:200]}...\n")
    
    # Stage 2: Editorial refinement
    if verbose:
        print("[Stage 2: Editorial Refinement]")
    rewrite = editor.act(
        f"Rewrite professionally with a 1-sentence executive summary first. "
        f"Improve clarity, keep bullet structure if present. Source:\n{research}"
    )
    if verbose:
        print(f"Output: {rewrite[:200]}...\n")
    
    return {
        'question': q,
        'research': research,
        'final': rewrite,
        'models': {
            'researcher': PRIMARY_MODEL_ID,
            'editor': EDITOR_MODEL_ID
        }
    }

# Execute sample pipeline
print("="*80)
result = pipeline('Explain why edge AI matters for compliance and latency.')
print("="*80)
print("\n[FINAL OUTPUT]")
print(result['final'])
print("\n[METADATA]")
print(f"Models used: {result['models']}")
result

[Pipeline] Question: Explain why edge AI matters for compliance and latency.

[Stage 1: Research]
Output: - **Data Sovereignty**: Edge AI allows data to be processed locally, which can help organizations comply with regional data protection regulations by keeping sensitive information within the borders o...

[Stage 2: Editorial Refinement]


### Επεξήγηση: Εκτέλεση & Αποτελέσματα Pipeline
Εκτελεί την πολυ-πρακτορική διαδικασία σε μια ερώτηση με θέμα τη συμμόρφωση και την καθυστέρηση για να δείξει:
- Πολυ-σταδιακή μετατροπή πληροφοριών
- Εξειδίκευση και συνεργασία πρακτόρων
- Βελτίωση της ποιότητας του αποτελέσματος μέσω επεξεργασίας
- Ιχνηλασιμότητα (διατηρούνται τόσο τα ενδιάμεσα όσο και τα τελικά αποτελέσματα)

**Δομή Αποτελέσματος:**
- `question` - Αρχικό ερώτημα χρήστη
- `research` - Ακατέργαστο ερευνητικό αποτέλεσμα (συνοπτικά γεγονότα)
- `final` - Επεξεργασμένη εκτελεστική σύνοψη
- `models` - Ποια μοντέλα χρησιμοποιήθηκαν σε κάθε στάδιο

**Ιδέες Επέκτασης:**
1. Προσθήκη ενός Πράκτορα Κριτικής για έλεγχο ποιότητας
2. Εφαρμογή παράλληλων πρακτόρων έρευνας για διαφορετικές πτυχές
3. Προσθήκη ενός Πράκτορα Επαλήθευσης για έλεγχο γεγονότων
4. Χρήση διαφορετικών μοντέλων για διαφορετικά επίπεδα πολυπλοκότητας
5. Εφαρμογή βρόχων ανατροφοδότησης για επαναληπτική βελτίωση


### Προχωρημένο: Προσαρμογή Ρύθμισης Πράκτορα

Δοκιμάστε να προσαρμόσετε τη συμπεριφορά του πράκτορα τροποποιώντας τις μεταβλητές περιβάλλοντος πριν εκτελέσετε το κελί αρχικοποίησης:

**Διαθέσιμα Μοντέλα:**
- Χρησιμοποιήστε `foundry model ls` στο τερματικό για να δείτε όλα τα διαθέσιμα μοντέλα
- Παραδείγματα: phi-4-mini, phi-3.5-mini, qwen2.5-7b, llama-3.2-3b, κ.λπ.


In [ ]:
# Example: Use different models for different agents
# Uncomment and modify as needed:

# import os
# os.environ['AGENT_MODEL_PRIMARY'] = 'phi-4-mini'      # Fast, good for research
# os.environ['AGENT_MODEL_EDITOR'] = 'qwen2.5-7b'       # Higher quality for editing

# Then restart the kernel and re-run all cells

# Test with different questions
test_questions = [
    "What are 3 key benefits of using small language models?",
    "How does RAG improve AI accuracy?",
    "Why is local inference important for privacy?"
]

print("Testing pipeline with multiple questions:\n")
for i, q in enumerate(test_questions, 1):
    print(f"\n{'='*80}")
    print(f"Question {i}: {q}")
    print('='*80)
    r = pipeline(q, verbose=False)
    print(f"\n[FINAL]: {r['final'][:300]}...")
    print(f"[Models]: Researcher={r['models']['researcher']}, Editor={r['models']['editor']}")


Testing pipeline with multiple questions:


Question 1: What are 3 key benefits of using small language models?

[FINAL]: <|channel|>analysis<|message|>The user wants a rewrite of the entire block of text. The rewrite should be professional, include a one-sentence executive summary first, improve clarity, keep bullet structure if present. The user has provided a large amount of text. The user wants a rewrite of that te...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 2: How does RAG improve AI accuracy?

[FINAL]: <|channel|>final<|message|>**RAG (Retrieval‑Augmented Generation) empowers AI to produce highly accurate, contextually relevant responses by combining a retrieval system with a large language model (LLM).**<|return|>...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 3: Why is local inference important for privacy?

[FINAL]: <|channel|>final<|message|>**Local inference—processing data d


---

**Αποποίηση ευθύνης**:  
Αυτό το έγγραφο έχει μεταφραστεί χρησιμοποιώντας την υπηρεσία μετάφρασης AI [Co-op Translator](https://github.com/Azure/co-op-translator). Παρόλο που καταβάλλουμε προσπάθειες για ακρίβεια, παρακαλούμε να έχετε υπόψη ότι οι αυτοματοποιημένες μεταφράσεις ενδέχεται να περιέχουν λάθη ή ανακρίβειες. Το πρωτότυπο έγγραφο στη μητρική του γλώσσα θα πρέπει να θεωρείται η αυθεντική πηγή. Για κρίσιμες πληροφορίες, συνιστάται επαγγελματική ανθρώπινη μετάφραση. Δεν φέρουμε ευθύνη για τυχόν παρεξηγήσεις ή εσφαλμένες ερμηνείες που προκύπτουν από τη χρήση αυτής της μετάφρασης.
